In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)

import numpy as np

In [3]:
import sys
sys.path.insert(0, "/home/storage/hans/jax_reco/python")
from geo import closest_distance_dom_track, closest_distance_dom_track_v
from geo import light_travel_time, light_travel_time_v
from geo import z_component_closest_point_on_track, z_component_closest_point_on_track
from geo import light_travel_time_i3calculator, light_travel_time_i3calculator_v
from geo import convert_spherical_to_cartesian_direction
from geo import cherenkov_cylinder_coordinates, cherenkov_cylinder_coordinates_v

In [4]:
from icecube import dataclasses
from icecube.phys_services import I3Calculator
# can use the following functions:
#    I3Calculator.cherenkov_time
#    I3Calculator.closest_approach_distance
#    I3Calculator.closest_approach_position
# example: double I3Calculator::CherenkovTime(const I3Particle& particle, 
# const I3Position& position, const double IndexRefG, const double IndexRefP )

from icecube.icetray import I3Units

In [5]:
# from icetray
# https://github.com/icecube/icetray/blob/main/dataclasses/public/dataclasses/I3Constants.h
#__n_ice_phase = 1.3195;
#__n_ice_group = 1.35634;
#__n_ice = __n_ice_group

# or use the values from clsim in our tables
#  'n_group': 1.32548384613875,
#  'n_phase': 1.30799291638281
__n_ice_phase_clsim = 1.30799291638281
__n_ice_group_clsim = 1.32548384613875
__n_ice_clsim = __n_ice_group_clsim

__n_ice_phase = __n_ice_phase_clsim
__n_ice_group = __n_ice_group_clsim
__n_ice = __n_ice_clsim

__theta_cherenkov = np.arccos(1/__n_ice_phase)
__c = 0.299792458 # m / ns
__c_ice = __c/__n_ice_group

# from Matti's code
# https://github.com/HansN87/gamma_mixture_photonics/blob/reconstruction/evaluation%20scripts/GetLLHs.py

_recip__speedOfLight = 3.3356409519815204
#_n__ = 1.32548384613875
#_tan__thetaC = (_n__**2.-1.)**0.5

# some comparisons point to difference in n_phase:
#print(np.tan(__theta_cherenkov), _tan__thetaC)
# 0.8608601802848123 0.8700042680210099

#print(__n_ice_phase, _n__)
# 1.3195 1.32548384613875

#print((__n_ice_phase**2.-1.)**0.5)
# 0.8608601802848125

In [6]:
# the icetray way ... ugh. ever heard the phrase "something being non-pythonic" ....
# will use these as point of comparison.

def get_icecube_track_i3particle(track_pos, track_dir):
    # track_pos: 3 component cartesian vector
    # track_dir: 2 component spherical vector (theta, phi)
    pos = dataclasses.I3Position(track_pos[0], track_pos[1], track_pos[2])
    d = dataclasses.I3Direction()
    d.set_theta_phi(track_dir[0], track_dir[1])
    pt = 0.0 # set vertex time to 0
    i3p = dataclasses.I3Particle(pos, d, pt, dataclasses.I3Particle.ParticleShape.InfiniteTrack, 100000 * I3Units.m)
    return i3p

def get_cherenkov_time(p, dom_pos):
    pos = dataclasses.I3Position(dom_pos[0] * I3Units.m, dom_pos[1] * I3Units.m, dom_pos[2] * I3Units.m)
    return I3Calculator.cherenkov_time(p, pos, __n_ice_group, __n_ice_phase)

def get_closest_approach_distance(p, dom_pos):
    pos = dataclasses.I3Position(dom_pos[0] * I3Units.m, dom_pos[1] * I3Units.m, dom_pos[2] * I3Units.m)
    return I3Calculator.closest_approach_distance(p, pos)

def get_closest_approach_position(p, dom_pos):
    pos = dataclasses.I3Position(dom_pos[0] * I3Units.m, dom_pos[1] * I3Units.m, dom_pos[2] * I3Units.m)
    return I3Calculator.closest_approach_position(p, pos)

In [7]:
def light_travel_time_eijndhoven(dom_pos, track_pos, track_dir):
    """
    eq. 3 / Fig. 1 of https://arxiv.org/pdf/0704.1706
    """
    v_a = dom_pos - track_pos

    closest_dist = closest_distance_dom_track(dom_pos, track_pos, track_dir)
    d1 = jnp.dot(v_a, track_dir)
    
    dt = _recip__speedOfLight * (d1 + closest_dist * (__n_ice_group_clsim * __n_ice_phase_clsim - 1) / jnp.sqrt(__n_ice_phase_clsim * __n_ice_phase_clsim -1))
    return dt

In [8]:
# single example:

track_pos = np.array([-10., 10., -200.])
track_theta = np.deg2rad(10.)
track_phi = np.deg2rad(2.)
track_dir_sph = np.array([track_theta, track_phi])
track_dir = np.array(convert_spherical_to_cartesian_direction(jnp.array(track_dir_sph)))

i3track = get_icecube_track_i3particle(track_pos, track_dir_sph)
dom_pos = np.array([0, 0, -150])

print(get_cherenkov_time(i3track, dom_pos)) # geo time
print(get_closest_approach_distance(i3track, dom_pos)) # closest distance to dom
print(get_closest_approach_position(i3track, dom_pos)[2]) # z component of closest position on track

199.95201012714827
10.375040484114887
-149.85830716519607


In [9]:
print(light_travel_time_i3calculator(dom_pos, track_pos, track_dir))

199.9520101271483


In [10]:
dt, s, z = cherenkov_cylinder_coordinates(dom_pos, track_pos, track_dir)
print(dt)
print(s)
print(z)

199.9520101271483
10.375040484114885
-149.85830716519607


In [11]:
print(light_travel_time(dom_pos, track_pos, track_dir)) # ~1.5 ns difference compared to icetray's I3Calculator.
print(closest_distance_dom_track(dom_pos, track_pos, track_dir))
print(z_component_closest_point_on_track(dom_pos, track_pos, track_dir))

199.94345145110827
10.375040484114885
-149.85830716519607


In [12]:
print(light_travel_time_eijndhoven(dom_pos, track_pos, track_dir))

199.95201012714827


In [13]:
# now demonstrate a vectorized example

track_pos = np.array([-10., 10., -200.])
track_theta = np.deg2rad(10.)
track_phi = np.deg2rad(2.)
track_dir_sph = np.array([track_theta, track_phi])
track_dir = np.array(convert_spherical_to_cartesian_direction(jnp.array(track_dir_sph)))

i3track = get_icecube_track_i3particle(track_pos, track_dir_sph)

n_doms = 50
dom_pos = np.random.normal(0.0, 300, (n_doms, 3))

# single call that computes everything over the n_doms matrix
dt, s, z = cherenkov_cylinder_coordinates_v(dom_pos, track_pos, track_dir)

In [14]:
# and the awkward icetray style loop
for i, d_pos in enumerate(dom_pos):
    dt_i3c = get_cherenkov_time(i3track, d_pos)
    print(f"delay time: {dt[i]}, {dt_i3c}") 

delay time: 1884.698515916533, 1884.698515916533
delay time: 3610.9923590396356, 3610.9923590396356
delay time: 1952.9559119763908, 1952.9559119763906
delay time: 1625.3913560175758, 1625.3913560175758
delay time: 1312.6635251187663, 1312.6635251187658
delay time: 2669.3316722180803, 2669.3316722180803
delay time: 4685.405048016598, 4685.405048016597
delay time: 1105.3329243723986, 1105.3329243723983
delay time: 1489.0334008130096, 1489.0334008130094
delay time: 2304.9776240589927, 2304.977624058992
delay time: 2303.3012914522583, 2303.3012914522583
delay time: 2377.84169706103, 2377.84169706103
delay time: 806.0388450138894, 806.0388450138897
delay time: 2459.3606370529005, 2459.3606370529005
delay time: 346.2364353508182, 346.23643535081817
delay time: 165.22969392609275, 165.22969392609255
delay time: 880.5243851171149, 880.5243851171148
delay time: -655.7543114653313, -655.7543114653317
delay time: 1446.2677382968805, 1446.2677382968802
delay time: 892.9454672512956, 892.9454672512

In [15]:
for i, d_pos in enumerate(dom_pos):
    s_i3c = get_closest_approach_distance(i3track, d_pos)
    print(f"closest approach distance: {s[i]}, {s_i3c}") 

closest approach distance: 365.3280630880061, 365.3280630880062
closest approach distance: 353.37933002789106, 353.37933002789106
closest approach distance: 243.3327190764864, 243.33271907648643
closest approach distance: 160.4315576282276, 160.43155762822758
closest approach distance: 29.61776122052171, 29.617761220521693
closest approach distance: 607.4053959237596, 607.4053959237598
closest approach distance: 919.6854408627439, 919.685440862744
closest approach distance: 488.9165235104298, 488.9165235104298
closest approach distance: 359.5653492286575, 359.56534922865745
closest approach distance: 315.10760739389957, 315.10760739389957
closest approach distance: 335.07301863799984, 335.07301863799984
closest approach distance: 386.50390125157537, 386.50390125157537
closest approach distance: 339.65671540227487, 339.65671540227487
closest approach distance: 781.4218034597084, 781.4218034597084
closest approach distance: 487.4320992576221, 487.43209925762216
closest approach distance:

In [16]:
for i, d_pos in enumerate(dom_pos):
    z_i3c = get_closest_approach_position(i3track, d_pos)[2]
    print(f"closest approach point (z-component): {z[i]}, {z_i3c}") 



closest approach point (z-component): 43.33720946112538, 43.337209461125354
closest approach point (z-component): 563.2450708989481, 563.2450708989481
closest approach point (z-component): 168.043112778644, 168.043112778644
closest approach point (z-component): 142.38245280864095, 142.38245280864095
closest approach point (z-component): 162.16474637231704, 162.16474637231698
closest approach point (z-component): 67.52302647849649, 67.52302647849649
closest approach point (z-component): 395.1108479884372, 395.1108479884372
closest approach point (z-component): -292.68015977952723, -292.68015977952723
closest approach point (z-component): -68.53932130531237, -68.53932130531234
closest approach point (z-component): 210.46000847653227, 210.4600084765322
closest approach point (z-component): 192.85412513965986, 192.85412513965986
closest approach point (z-component): 170.78344573184728, 170.78344573184728
closest approach point (z-component): -253.1229185180843, -253.12291851808433
closest 